### Pretrain in Dictprompt

In [1]:
lr = 3e-5
wd = 1e-6
batch_size = 8
epoch = 5
#path = "wordgamesBERTbase3kGM"
template_id = 7
path = "wgsBERTbase3kT{}".format(template_id)

In [2]:
model_name="F:\\preTrainedModels\\bert-base-uncased\\"

# 1.0 load dataset

In [3]:
import pandas as pd
import re

In [4]:
train = pd.read_json('WordGameData/train_single_wordgames_ox3k.json')
val = pd.read_json('WordGameData/val_single_wordgames_ox3k.json')

In [5]:
train

,word,mean,sentence
20662,proceed,to continue doing sth that has already been st...,We're not sure whether we still want to proce...
9356,extension,a new part that is added to a building,a planned two-storey extension to the hospital
6533,cup,to make your hands into the shape of a bowl,She held the bird gently in cupped hands.
19993,point,a particular detail or fact,Here are the main points of the news.
15999,love,to have very strong feelings of affection for sb,"If you love each other, why not get married?"
...,...,...,...
18702,pack,to do a lot of things in a limited period of time,You seem to have packed a lot into your life!
7130,deny,to refuse to admit or accept sth,She denied all knowledge of the incident.
14493,itself,"used to emphasize an animal, a thing, etc.","The village itself is pretty, but the surroun..."
10843,free,not attached to sth or trapped by sth,Pull gently on the free end of the rope.


In [6]:
train['mean'].map(lambda x : len(x.split(' '))).mean()

11.971776459569154

# 1.1 templates definition

In [7]:
from datasets import Dataset
from transformers import BertTokenizer

In [8]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [9]:
tokenizer.mask_token

'[MASK]'

In [10]:
def process_data_template_0(df_row_data):
    len_word = len(tokenizer.encode(df_row_data['word'],add_special_tokens=False))
    templated_sentence= " ".join([tokenizer.mask_token for i in range(len_word)]) + " means " + df_row_data['mean']
    label = df_row_data['word']
    return templated_sentence,label

def process_data_template_1(df_row_data):
    len_mean = len(tokenizer.encode(df_row_data['mean'],add_special_tokens=False))
    templated_sentence= df_row_data['word'] + " means " + " ".join([tokenizer.mask_token for i in range(len_mean)])
    label=df_row_data['mean']
    return templated_sentence,label

In [11]:
def process_data_template_5(df_row_data):
    # The { word } in sentence { example } means { meaning }
    len_word = len(tokenizer.encode(df_row_data['word'],add_special_tokens=False))
    len_mean = len(tokenizer.encode(df_row_data['mean'],add_special_tokens=False))
    len_label_1 = len(tokenizer.encode("The word ",add_special_tokens=False))
    len_label_2 = len(tokenizer.encode(" in sentence '",add_special_tokens=False))
    
    word_list = df_row_data['sentence'].split(' ')
    # 先构建原始句子
    template5_sentence = "The word " + " ".join([tokenizer.mask_token for i in range(len_word)]) + " in sentence '"
    label = " ".join([tokenizer.mask_token for i in range(len_label_1)]) +  " " + df_row_data['word'] + " " + " ".join([tokenizer.mask_token for i in range(len_label_2)])
    sub_sentence_list = []
    temp_word_list = []
    label_word_list = []
    #temp_w_num = 0
    #print(word_list)
    #print()
    # 做这个处理的原因是，有可能一个句子里存在多个目标词
    for w in word_list:
        if re.match(df_row_data['word'],w) is not None:
            #print("match")
            #print(temp_word_list)
            label_word_list.append(w)
            if temp_word_list == []:
                continue
            sub_sentence_list.append(" ".join(temp_word_list))
            temp_word_list = []
            #temp_w_num += 1
        else:
            temp_word_list.append(w)
            #print(temp_word_list)
    sub_sentence_list.append(" ".join(temp_word_list))
    #print(label_word_list)
    # 这样做的目的是，有些词的不同形式可能存在不同的tokenizer的大小
    
    w_num = sum([len(tokenizer.encode(i,add_special_tokens=False)) for i in label_word_list]) #标签中的词被处理后的id数量
    sent_num = 0 # template句子中的词的数量
    temp_w_num = 0 # 用于存储每个词的id累积到什么程度
    #temp_sent_num = 0
    #print(sub_sentence_list)
    label += " "
    # 把词都填回去
    for sub_sent in sub_sentence_list:
        template5_sentence += sub_sent
        template5_sentence += " "
        temp_sent_num = len(tokenizer.encode(sub_sent,add_special_tokens=False))
        label += " ".join([tokenizer.mask_token for i in range(temp_sent_num)])
        sent_num += temp_sent_num
        if temp_w_num < w_num:
            w = label_word_list.pop(0)
            temp_w_num += len(tokenizer.encode(w,add_special_tokens=False))
            template5_sentence += " ".join([tokenizer.mask_token for i in range(temp_w_num)])
            template5_sentence += " "
            label += " "
            label += w
    label += " "
    len_label_3 = len(tokenizer.encode("' means '",add_special_tokens=False))
    label += " ".join([tokenizer.mask_token for i in range(len_label_3)])
    label += " "
    label += df_row_data['mean']
    label += " "
    label += " ".join([tokenizer.mask_token for i in range(len(tokenizer.encode("'.",add_special_tokens=False)))])
    #print(sub_sentence_list)
    # 组成最终的句子
    template5_sentence = template5_sentence + "' means '" + " ".join([tokenizer.mask_token for i in range(len_mean)]) + "'."
    return template5_sentence,label

In [12]:
def process_data_template6(df_row_data):
    #{ meaning } is the meaning for {word} in {example}
    label = ""
    
    sub_temp_0 = "'"
    len_sub_temp_0 = len(tokenizer.encode(sub_temp_0,add_special_tokens=False))
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_0)])
    
    len_mean = len(tokenizer.encode(df_row_data['mean'],add_special_tokens=False))
    label += " "
    label += df_row_data['mean']
    
    sub_temp_1 = "' is the meaning for word '"
    len_sub_temp_1 = len(tokenizer.encode(sub_temp_1,add_special_tokens=False))
    template6_sentence =  sub_temp_0 + " ".join([tokenizer.mask_token for i in range(len_mean)]) + sub_temp_1 # 拼接模板后的子句
    label += " "
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_1)])
    
    len_word = len(tokenizer.encode(df_row_data['word'],add_special_tokens=False))
    template6_sentence += " ".join([tokenizer.mask_token for i in range(len_word)])
    label += " "
    label += df_row_data['word']
    
    sub_temp_2 = "' in sentence '"
    len_sub_temp_2 = len(tokenizer.encode(sub_temp_2,add_special_tokens=False))
    template6_sentence += sub_temp_2#"
    label += " "
    label +=  " ".join([tokenizer.mask_token for i in range(len_sub_temp_2)])
    
    #加入句子
    word_list = df_row_data['sentence'].split(' ')
    
    sub_sentence_list = []
    temp_word_list = []
    label_word_list = []

    # 做这个处理的原因是，有可能一个句子里存在多个目标词
    for w in word_list:
        if re.match(df_row_data['word'],w) is not None:
            #print("match")
            #print(temp_word_list)
            label_word_list.append(w)
            if temp_word_list == []:
                continue
            sub_sentence_list.append(" ".join(temp_word_list))
            temp_word_list = []
            #temp_w_num += 1
        else:
            temp_word_list.append(w)
            #print(temp_word_list)
    sub_sentence_list.append(" ".join(temp_word_list))
    
    w_num = sum([len(tokenizer.encode(i,add_special_tokens=False)) for i in label_word_list]) #标签中的词被处理后的id数量
    sent_num = 0 # template句子中的词的数量
    temp_w_num = 0 # 用于存储每个词的id累积到什么程度
    #temp_sent_num = 0
    #print(sub_sentence_list)
    label += " "
    
    # 把词都填回去
    #print(len_mean)
    for sub_sent in sub_sentence_list:
        template6_sentence += sub_sent
        template6_sentence += " "
        temp_sent_num = len(tokenizer.encode(sub_sent,add_special_tokens=False))
        label += " ".join([tokenizer.mask_token for i in range(temp_sent_num)])
        sent_num += temp_sent_num
        if temp_w_num < w_num:
            w = label_word_list.pop(0)
            temp_w_num += len(tokenizer.encode(w,add_special_tokens=False))
            #template6_sentence += " "
            template6_sentence += " ".join([tokenizer.mask_token for i in range(temp_w_num)])
            template6_sentence += " "
            label += " "
            label += w
            label += " "
    sub_temp_3 = "'"
    template6_sentence += sub_temp_3
    return template6_sentence,label

In [13]:
def process_data_template_7(df_row_data):
    # the word {word} means {meaning}, for example in sentence {sentence}.
    # 先是一些预备的东西，用于连接句子
    sub_temp_0 = "The word '"
    len_sub_temp_0 = len(tokenizer.encode(sub_temp_0,add_special_tokens=False))
    len_word = len(tokenizer.encode(df_row_data['word'],add_special_tokens=False))
    
    sub_temp_1 = "' means '"
    len_sub_temp_1 = len(tokenizer.encode(sub_temp_1,add_special_tokens=False))
    
    len_mean = len(tokenizer.encode(df_row_data['mean'],add_special_tokens=False))
    
    sub_temp_2 = "', for example in sentence '"
    len_sub_temp_2 = len(tokenizer.encode(sub_temp_2,add_special_tokens=False))

    sub_temp_3 = "'"
    len_sub_temp_3 = len(tokenizer.encode(sub_temp_3,add_special_tokens=False))

    template_sentence_7 = ""
    label = ""
    # 开始拼接句子
    # 先把简单的拼上去
    template_sentence_7 += sub_temp_0
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_0)])
    label += " "
    template_sentence_7 += " ".join([tokenizer.mask_token for i in range(len_word)])
    label += df_row_data['word']
    label += " "
    
    template_sentence_7 += sub_temp_1
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_1)])
    label += " "
    template_sentence_7 += " ".join([tokenizer.mask_token for i in range(len_mean)])
    label += df_row_data['mean']
    label += " "
    
    template_sentence_7 += sub_temp_2
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_2)])
    label += " "
    #template_sentence_7 += 
    
    #加入句子
    word_list = df_row_data['sentence'].split(' ')
    
    sub_sentence_list = []
    temp_word_list = []
    label_word_list = []

    # 做这个处理的原因是，有可能一个句子里存在多个目标词
    for w in word_list:
        if re.match(df_row_data['word'],w) is not None:
            #print("match")
            #print(temp_word_list)
            label_word_list.append(w)
            if temp_word_list == []:
                continue
            sub_sentence_list.append(" ".join(temp_word_list))
            temp_word_list = []
            #temp_w_num += 1
        else:
            temp_word_list.append(w)
            #print(temp_word_list)
    sub_sentence_list.append(" ".join(temp_word_list))
    
    w_num = sum([len(tokenizer.encode(i,add_special_tokens=False)) for i in label_word_list]) #标签中的词被处理后的id数量
    sent_num = 0 # template句子中的词的数量
    temp_w_num = 0 # 用于存储每个词的id累积到什么程度
    #temp_sent_num = 0
    #print(sub_sentence_list)
    label += " "
    
    # 把词都填回去
    #print(len_mean)
    for sub_sent in sub_sentence_list:
        template_sentence_7 += sub_sent
        template_sentence_7 += " "
        temp_sent_num = len(tokenizer.encode(sub_sent,add_special_tokens=False))
        label += " ".join([tokenizer.mask_token for i in range(temp_sent_num)])
        sent_num += temp_sent_num
        if temp_w_num < w_num:
            w = label_word_list.pop(0)
            temp_w_num += len(tokenizer.encode(w,add_special_tokens=False))
            #template6_sentence += " "
            template_sentence_7 += " ".join([tokenizer.mask_token for i in range(temp_w_num)])
            template_sentence_7 += " "
            label += " "
            label += w
            label += " "
    template_sentence_7 += sub_temp_3
    label += " "
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_3)])
    return template_sentence_7,label

In [ ]:
def process_data_template_8(df_row_data):
    # In sentence {sentence}, the interpretation of the {word} is {meaning}
    sub_temp_0 = "In sentence '"
    sub_temp_1 = "', the interpretation of the word '"
    sub_temp_2 = "' is '"
    sub_temp_3 = "'."
    
    len_sub_temp_0 = len(tokenizer.encode(sub_temp_0,add_special_tokens=False))
    len_sub_temp_1 = len(tokenizer.encode(sub_temp_1,add_special_tokens=False))
    len_sub_temp_2 = len(tokenizer.encode(sub_temp_2,add_special_tokens=False))
    len_sub_temp_3 = len(tokenizer.encode(sub_temp_3,add_special_tokens=False))
    
    len_word = len(tokenizer.encode(df_row_data['word'],add_special_tokens=False))
    len_mean = len(tokenizer.encode(df_row_data['mean'],add_special_tokens=False))
    
    template_sentence_8 = ""
    label = " "
    
    template_sentence_8 += sub_temp_0
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_0)])
    
    #加入句子
    word_list = df_row_data['sentence'].split(' ')
    
    sub_sentence_list = []
    temp_word_list = []
    label_word_list = []

    # 做这个处理的原因是，有可能一个句子里存在多个目标词
    for w in word_list:
        if re.match(df_row_data['word'],w) is not None:
            #print("match")
            #print(temp_word_list)
            label_word_list.append(w)
            if temp_word_list == []:
                continue
            sub_sentence_list.append(" ".join(temp_word_list))
            temp_word_list = []
            #temp_w_num += 1
        else:
            temp_word_list.append(w)
            #print(temp_word_list)
    sub_sentence_list.append(" ".join(temp_word_list))
    
    w_num = sum([len(tokenizer.encode(i,add_special_tokens=False)) for i in label_word_list]) #标签中的词被处理后的id数量
    sent_num = 0 # template句子中的词的数量
    temp_w_num = 0 # 用于存储每个词的id累积到什么程度
    #temp_sent_num = 0
    #print(sub_sentence_list)
    label += " "
    
    # 把词都填回去
    #print(len_mean)
    for sub_sent in sub_sentence_list:
        template_sentence_8 += sub_sent
        template_sentence_8 += " "
        temp_sent_num = len(tokenizer.encode(sub_sent,add_special_tokens=False))
        label += " ".join([tokenizer.mask_token for i in range(temp_sent_num)])
        sent_num += temp_sent_num
        if temp_w_num < w_num:
            w = label_word_list.pop(0)
            temp_w_num += len(tokenizer.encode(w,add_special_tokens=False))
            #template6_sentence += " "
            template_sentence_8 += " ".join([tokenizer.mask_token for i in range(temp_w_num)])
            template_sentence_8 += " "
            label += " "
            label += w
            label += " "    
    
    template_sentence_8 += sub_temp_1
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_1)])
    label += " "
    
    template_sentence_8 += " ".join([tokenizer.mask_token for i in range(len_word)])
    label += df_row_data['word']
    label += " "
    
    template_sentence_8 += sub_temp_2
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_2)])
    label += " "
    
    template_sentence_8 += " ".join([tokenizer.mask_token for i in range(len_mean)])
    label += df_row_data['mean']
    label += " "
    
    template_sentence_8 += sub_temp_3
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_3)])
    return template_sentence_8,label

In [ ]:
def process_data_template_9(df_row_data):
    # In the sentence {sentence}, word that means {meaning} is {word}
    sub_temp_0 = "In the sentence '"
    sub_temp_1 = "', word that means '"
    sub_temp_2 = "' is '"
    sub_temp_3 = "'."
    
    
    len_sub_temp_0 = len(tokenizer.encode(sub_temp_0,add_special_tokens=False))
    len_sub_temp_1 = len(tokenizer.encode(sub_temp_1,add_special_tokens=False))
    len_sub_temp_2 = len(tokenizer.encode(sub_temp_2,add_special_tokens=False))
    len_sub_temp_3 = len(tokenizer.encode(sub_temp_3,add_special_tokens=False))
    
    len_word = len(tokenizer.encode(df_row_data['word'],add_special_tokens=False))
    len_mean = len(tokenizer.encode(df_row_data['mean'],add_special_tokens=False))
    
    template_sentence_9 = ""
    label = " "
    
    template_sentence_9 += sub_temp_0
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_0)])
    label += " "
    
    #加入句子
    word_list = df_row_data['sentence'].split(' ')
    
    sub_sentence_list = []
    temp_word_list = []
    label_word_list = []

    # 做这个处理的原因是，有可能一个句子里存在多个目标词
    for w in word_list:
        if re.match(df_row_data['word'],w) is not None:
            #print("match")
            #print(temp_word_list)
            label_word_list.append(w)
            if temp_word_list == []:
                continue
            sub_sentence_list.append(" ".join(temp_word_list))
            temp_word_list = []
            #temp_w_num += 1
        else:
            temp_word_list.append(w)
            #print(temp_word_list)
    sub_sentence_list.append(" ".join(temp_word_list))
    
    w_num = sum([len(tokenizer.encode(i,add_special_tokens=False)) for i in label_word_list]) #标签中的词被处理后的id数量
    sent_num = 0 # template句子中的词的数量
    temp_w_num = 0 # 用于存储每个词的id累积到什么程度
    #temp_sent_num = 0
    #print(sub_sentence_list)
    label += " "
    
    # 把词都填回去
    #print(len_mean)
    for sub_sent in sub_sentence_list:
        template_sentence_9 += sub_sent
        template_sentence_9 += " "
        temp_sent_num = len(tokenizer.encode(sub_sent,add_special_tokens=False))
        label += " ".join([tokenizer.mask_token for i in range(temp_sent_num)])
        sent_num += temp_sent_num
        if temp_w_num < w_num:
            w = label_word_list.pop(0)
            temp_w_num += len(tokenizer.encode(w,add_special_tokens=False))
            #template6_sentence += " "
            template_sentence_9 += " ".join([tokenizer.mask_token for i in range(temp_w_num)])
            template_sentence_9 += " "
            label += " "
            label += w
            label += " "    
    
    label += " "
    template_sentence_9 += sub_temp_1
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_1)])
    label += " "
    
    template_sentence_9 += " ".join([tokenizer.mask_token for i in range(len_mean)])
    label += df_row_data['mean']
    label += " "
    
    template_sentence_9 += sub_temp_2
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_2)])
    label += " "
    
    template_sentence_9 += " ".join([tokenizer.mask_token for i in range(len_word)])
    label += df_row_data['word']
    label += " "
    
    template_sentence_9 += sub_temp_3
    label += " ".join([tokenizer.mask_token for i in range(len_sub_temp_3)])
    label += " "
    
    return template_sentence_9,label

In [14]:
def process_df_row_context_func(df_row_data,template_id):
    # 直接是词语对应意思的方式
    if template_id == 0:
        templated_sentence,label = process_data_template_0(df_row_data)
    elif template_id == 1:
        templated_sentence,label = process_data_template_1(df_row_data)
    elif template_id == 2:
        # 根据上下文，生成词语和意思,在句子中间加带有括号的解释
        word_list = df_row_data['sentence'].split(' ')
        templated_sentence_list = []
        label_list = []
        for w in word_list:
            if tokenizer(w,add_special_tokens=False) == tokenizer(df_row_data['word'],add_special_tokens=False):
                append_mean = w + " ( which means {} )".format(df_row_data['mean'])
                #label_list.append(w)
                label_list.append(append_mean)
                templated_sentence_list += ["[MASK]"] * len(tokenizer.encode(append_mean,add_special_tokens=False))
            else:
                templated_sentence_list.append(w)
                #label_list.append("[MASK]")
        templated_sentence = " ".join(templated_sentence_list)
        label = " ".join(label_list)
    elif template_id == 3:
        # 根据上下文，生成词语和意思,在句子中间加带有括号的解释,括号里的更少了
        word_list = df_row_data['sentence'].split(' ')
        templated_sentence_list = []
        label_list = []
        for w in word_list:
            if tokenizer(w,add_special_tokens=False) == tokenizer(df_row_data['word'],add_special_tokens=False):
                append_mean = w + " ( {} )".format(df_row_data['mean'])
                #label_list.append(w)
                label_list.append(append_mean)
                templated_sentence_list += ["[MASK]"] * len(tokenizer.encode(append_mean,add_special_tokens=False))
            else:
                templated_sentence_list.append(w)
                #label_list.append("[MASK]")
        templated_sentence = " ".join(templated_sentence_list)
        label = " ".join(label_list)
    elif template_id == 4:
        # 根据上下文，生成词语和意思,但是改变了生成方式
        word_list = df_row_data['sentence'].split(' ')
        templated_sentence_list = []
        label_list = []
        for w in word_list:
            if tokenizer(w,add_special_tokens=False) == tokenizer(df_row_data['word'],add_special_tokens=False):
                label_list.append(w)
                templated_sentence_list.append("[MASK]")
            else:
                templated_sentence_list.append(w)
        append_mean = ", which {} means {} .".format(df_row_data['word'],df_row_data['mean'])
        label_list.append(append_mean)
        templated_sentence_list += ["[MASK]"] * len(tokenizer.encode(append_mean,add_special_tokens=False))
        templated_sentence = " ".join(templated_sentence_list)
        label = " ".join(label_list)
    elif template_id == 5:
        templated_sentence,label = process_data_template_5(df_row_data)
    elif template_id == 6:
        templated_sentence,label = process_data_template6(df_row_data)
    elif template_id == 7:
        templated_sentence,label = process_data_template_7(df_row_data)
    elif template_id == 8:
        templated_sentence,label = process_data_template_8(df_row_data)
    elif template_id == 9:
        templated_sentence,label = process_data_template_9(df_row_data)
    else:
        templated_sentence = ""
        label = ""
    return templated_sentence,label

In [36]:
templated_sentence,label = process_df_row_context_func(train.iloc[77],template_id)

In [37]:
templated_sentence,label

("The word '[MASK]' means '[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]', for example in sentence 'More and more travelers are looking for bed and breakfast [MASK] in private homes. '",
 '[MASK] [MASK] [MASK] accommodation [MASK] [MASK] [MASK] somewhere to live or stay, often also providing food or other services [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]  [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] accommodations [MASK] [MASK] [MASK] [MASK] [MASK]')

# 1.2 process dataset

In [17]:
def tokenize_function(examples,max_length=24,template_id = template_id):
    templated_sentence,label = process_df_row_context_func(examples,template_id)
    tokenized_sentence = tokenizer(templated_sentence,max_length = max_length, padding='max_length', truncation=True)
    label = tokenizer.encode(label,add_special_tokens=False)
    labels = []
    j = 0
    for i in tokenized_sentence.input_ids:
        if i != 103:
            labels.append(-100)
        else:
            labels.append(label[j])
            j += 1
    #labels = [-100 for i in tokenized_sentence.input_ids if i != 103 else label[0]]
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

In [18]:
tokenize_function(train.iloc[0])

{'input_ids': [101, 1996, 2773, 1005, 103, 1005, 2965, 1005, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, 103, -100, -100, -100, 103, 103, 10838, 103, 103, 103, 2000, 3613, 2725, 2358, 2232, 2008, 2038, 2525, 2042, -100]}

In [19]:
dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(val)

In [20]:
tokenized_train_datasets = dataset_train.map(tokenize_function, batched=False)
tokenized_test_datasets = dataset_test.map(tokenize_function, batched=False)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [21]:
tokenized_train_datasets = tokenized_train_datasets.remove_columns(['word','mean','sentence','__index_level_0__'])

In [22]:
for i in tokenized_train_datasets:
    print(i)
    break

{'input_ids': [101, 1996, 2773, 1005, 103, 1005, 2965, 1005, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, 103, -100, -100, -100, 103, 103, 10838, 103, 103, 103, 2000, 3613, 2725, 2358, 2232, 2008, 2038, 2525, 2042, -100]}


# 1.3 Load model  and test

In [23]:
import torch

In [24]:
from torch import nn
from torch.nn import CrossEntropyLoss
from transformers import (
    BertPreTrainedModel,
    BertModel
)
from transformers.modeling_outputs import (
    MaskedLMOutput
)
from transformers.models.bert.modeling_bert import (
    BertOnlyMLMHead
)

from typing import List, Optional, Tuple, Union


In [25]:
class BertForMaskedLM(BertPreTrainedModel):

    _keys_to_ignore_on_load_unexpected = [r"pooler"]
    _keys_to_ignore_on_load_missing = [r"position_ids", r"predictions.decoder.bias"]

    def __init__(self, config):
        super().__init__(config)

        if config.is_decoder:
            logger.warning(
                "If you want to use `BertForMaskedLM` make sure `config.is_decoder=False` for "
                "bi-directional self-attention."
            )

        self.bert = BertModel(config, add_pooling_layer=False)
        self.cls = BertOnlyMLMHead(config)

        # Initialize weights and apply final processing
        self.post_init()

    def get_output_embeddings(self):
        return self.cls.predictions.decoder

    def set_output_embeddings(self, new_embeddings):
        self.cls.predictions.decoder = new_embeddings

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], MaskedLMOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should be in `[-100, 0, ...,
            config.vocab_size]` (see `input_ids` docstring) Tokens with indices set to `-100` are ignored (masked), the
            loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`
        """

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]
        prediction_scores = self.cls(sequence_output)

        masked_lm_loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()  # -100 index = padding token
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))
        
        ## my outputs
        index_mask = (labels>0).nonzero(as_tuple=True)
        predictions = prediction_scores[index_mask]
        
        if not return_dict:
            output = (prediction_scores,) + outputs[2:]
            return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output

        return MaskedLMOutput(
            loss=masked_lm_loss,
            logits=predictions.argmax(axis=-1),
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def prepare_inputs_for_generation(self, input_ids, attention_mask=None, **model_kwargs):
        input_shape = input_ids.shape
        effective_batch_size = input_shape[0]

        #  add a dummy token
        if self.config.pad_token_id is None:
            raise ValueError("The PAD token should be defined for generation")

        attention_mask = torch.cat([attention_mask, attention_mask.new_zeros((attention_mask.shape[0], 1))], dim=-1)
        dummy_token = torch.full(
            (effective_batch_size, 1), self.config.pad_token_id, dtype=torch.long, device=input_ids.device
        )
        input_ids = torch.cat([input_ids, dummy_token], dim=1)

        return {"input_ids": input_ids, "attention_mask": attention_mask}

In [26]:
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at F:\preTrainedModels\bert-base-uncased\ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
o1 = model(
    input_ids=torch.LongTensor([i['input_ids']]),
    token_type_ids = torch.LongTensor(i['token_type_ids']),
    attention_mask=torch.LongTensor([i['attention_mask']]),
    labels=torch.LongTensor([i['labels']]),
)

In [28]:
o1

MaskedLMOutput(loss=tensor(11.7715, grad_fn=<NllLossBackward>), logits=tensor([1037, 2000, 1010, 1005, 1005, 1010, 1010, 1010, 1010, 1010, 1010, 1010,
        1005, 1005, 1005, 1012]), hidden_states=None, attentions=None)

In [29]:
model = model.cuda()

# 1.4 Train and Save

In [30]:
from transformers import Trainer,TrainingArguments

In [31]:
training_args = TrainingArguments(
    output_dir=path,
    evaluation_strategy="epoch",
    #eval_steps = 1,
    learning_rate         = lr,
    save_strategy='epoch',
    weight_decay          = wd,
    num_train_epochs      = epoch,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    #label_smoothing_factor = 0.01
)

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset =tokenized_test_datasets,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

In [33]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: word, mean, __index_level_0__, sentence. If word, mean, __index_level_0__, sentence are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1779
  Batch size = 8


{'eval_loss': 11.745624542236328,
 'eval_runtime': 4.4605,
 'eval_samples_per_second': 398.832,
 'eval_steps_per_second': 49.994}

In [34]:
trainer.train()